话说按照原始径向基函数那个定义，明显会造成一个feature冲掉另一个feature的影响（scale不一致），而为了使用libsvm内部的rbf机制，只能我自己先标准化了。y也要标准化，然后回归SVR过的值。这个对线性回归倒好像没有影响，记得计量经济学就讨论过先标准化对结果有什么影响。

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import os

from empirical_formulas import apply_formulas, func_name_list


In [2]:
df = pd.read_excel('GFR-SVR数据.xls')
df.columns = ['id', 'age', 'sex', 'rGFR', 'Scr', 'Cys']
df.shape

(1197, 6)

In [3]:
df['Cys'] = pd.to_numeric(df['Cys'], errors='coerce')
df=df.dropna()
df.shape

(1188, 6)

In [4]:
df = df[(18<df['age']) & (df['age']<=100) & (1<=df['sex']) & (df['sex']<=2) & (5 <= df['rGFR']) & (df['rGFR'] <= 150) & \
       (0.0 <= df['Scr']) & (df['Scr'] <= 3000) & (0.2 <= df['Cys']) & (df['Cys'] < 5.0)]
df.shape

(1081, 6)

In [5]:
mean = df[['rGFR','age', 'Scr', 'Cys']].mean()

In [6]:
mean

rGFR    66.467523
age     55.328400
Scr      1.640316
Cys      1.633626
dtype: float64

In [7]:
std = df[['rGFR','age', 'Scr', 'Cys']].std()
std

rGFR    29.007529
age     16.230584
Scr      1.529761
Cys      0.970992
dtype: float64

In [8]:
dfs = (df[['rGFR', 'age', 'Scr', 'Cys']] - mean)/std

In [9]:
dfs['sex'] = df['sex']

In [10]:
dfs.mean()

rGFR   -4.005430e-16
age    -6.732203e-17
Scr     3.380994e-16
Cys     1.232440e-17
sex     1.356152e+00
dtype: float64

In [11]:
dfs.std()

rGFR    1.000000
age     1.000000
Scr     1.000000
Cys     1.000000
sex     0.479082
dtype: float64

In [12]:
X = dfs[['age', 'Scr', 'Cys']]
y = dfs['rGFR']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

In [14]:
svr = SVR(kernel='rbf') # all deafult
svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [15]:
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)



In [16]:
print('y_train_pred(normalized)', np.mean((y_train_pred - y_train)**2))

y_train_pred(normalized) 0.23368467035385598


In [17]:
print('y_train_pred(normalized)', np.mean(((std['rGFR']*y_train_pred + mean['rGFR']) - (std['rGFR']*y_train + mean['rGFR']))**2))

y_train_pred(normalized) 196.6308686160882


In [18]:
print('y_test_pred(normalized)', np.mean((y_test_pred - y_test)**2))

y_test_pred(normalized) 0.221726594411519


In [19]:
print('y_test_pred(normalized)', np.mean(((std['rGFR']*y_test_pred + mean['rGFR']) - (std['rGFR']*y_test + mean['rGFR']))**2))

y_test_pred(normalized) 186.56890410657033


In [ ]:
'''
svr_rbf = SVR(kernel='rbf', C=100, gamma='auto', epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
'''
svr_rbf = SVR(kernel='rbf')
svr_lin = SVR(kernel='linear')
svr_poly = SVR(kernel='poly')

y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)


In [22]:
for method in ['rbf', 'linear', 'poly']:
    model = SVR(kernel = method)
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred =  model.predict(X_test)

    train_loss = np.mean((y_train -y_train_pred)**2)
    test_loss = np.mean((y_test - y_test_pred)**2)

    train_loss_re = np.mean(((std['rGFR']*y_train_pred + mean['rGFR']) - (std['rGFR']*y_train + mean['rGFR']))**2)
    test_loss_re =  np.mean(((std['rGFR']*y_test_pred + mean['rGFR']) - (std['rGFR']*y_test + mean['rGFR']))**2)

    print('method = {} tr_l ={:.4f} te_l = {:.4f} tr_lr = {:.4f} te_lr = {:.4f}'.format(method, \
                                train_loss, test_loss, train_loss_re, test_loss_re))


method = rbf tr_l =0.2337 te_l = 0.2217 tr_lr = 196.6309 te_lr = 186.5689
method = linear tr_l =0.3465 te_l = 0.3335 tr_lr = 291.5839 te_lr = 280.6250
method = poly tr_l =0.6180 te_l = 0.5355 tr_lr = 519.9700 te_lr = 450.6211


In [21]:
mat = np.empty([4,3,3,2])
for i,gamma in enumerate([0.1,1.0, 10.0, 'auto']):
    for j,C in enumerate([0.1,1.0,10]):
        for k,epsilon in enumerate([0.05,0.1,0.15]):
            svr_rbf_log_sex_test = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            svr_rbf_log_sex_test.fit(X_train, y_train)
            
            y_train_pred = svr_rbf_log_sex_test.predict(X_train)
            y_test_pred =  svr_rbf_log_sex_test.predict(X_test)
            
            train_loss = np.mean((y_train -y_train_pred)**2)
            test_loss = np.mean((y_test - y_test_pred)**2)
            
            train_loss_re = np.mean(((std['rGFR']*y_train_pred + mean['rGFR']) - (std['rGFR']*y_train + mean['rGFR']))**2)
            test_loss_re =  np.mean(((std['rGFR']*y_test_pred + mean['rGFR']) - (std['rGFR']*y_test + mean['rGFR']))**2)
            
            print('ga = {} C = {} eps = {} tr_l ={:.4f} te_l = {:.4f} tr_lr = {:.4f} te_lr = {:.4f}'.format(gamma, C, epsilon, \
                                        train_loss, test_loss, train_loss_re, test_loss_re))
            mat[i,j,k,0] = train_loss
            mat[i,j,k,1] = test_loss

ga = 0.1 C = 0.1 eps = 0.05 tr_l =0.2690 te_l = 0.2509 tr_lr = 226.3662 te_lr = 211.1428
ga = 0.1 C = 0.1 eps = 0.1 tr_l =0.2689 te_l = 0.2505 tr_lr = 226.2250 te_lr = 210.7486
ga = 0.1 C = 0.1 eps = 0.15 tr_l =0.2697 te_l = 0.2513 tr_lr = 226.9285 te_lr = 211.4691
ga = 0.1 C = 1.0 eps = 0.05 tr_l =0.2382 te_l = 0.2224 tr_lr = 200.4690 te_lr = 187.1667
ga = 0.1 C = 1.0 eps = 0.1 tr_l =0.2379 te_l = 0.2223 tr_lr = 200.1583 te_lr = 187.0890
ga = 0.1 C = 1.0 eps = 0.15 tr_l =0.2378 te_l = 0.2230 tr_lr = 200.1244 te_lr = 187.6227
ga = 0.1 C = 10 eps = 0.05 tr_l =0.2334 te_l = 0.2195 tr_lr = 196.3819 te_lr = 184.6538
ga = 0.1 C = 10 eps = 0.1 tr_l =0.2331 te_l = 0.2207 tr_lr = 196.1441 te_lr = 185.7432
ga = 0.1 C = 10 eps = 0.15 tr_l =0.2335 te_l = 0.2235 tr_lr = 196.5160 te_lr = 188.0779
ga = 1.0 C = 0.1 eps = 0.05 tr_l =0.2561 te_l = 0.2354 tr_lr = 215.4772 te_lr = 198.0930
ga = 1.0 C = 0.1 eps = 0.1 tr_l =0.2564 te_l = 0.2356 tr_lr = 215.7697 te_lr = 198.2329
ga = 1.0 C = 0.1 eps = 0.15 

In [51]:
print('{:.4f}'.format(np.nan))

nan


In [54]:
def grid_search(gamma_list, C_list, epsilon_list, kernel = 'rbf', verbose=True, y_origin =False):
    mat = np.empty([len(gamma_list), len(C_list), len(epsilon_list), 4])
    r_list = []
    for i,gamma in enumerate(gamma_list):
        for j,C in enumerate(C_list):
            for k,epsilon in enumerate(epsilon_list):
                model = SVR(kernel = kernel, C = C, gamma = gamma, epsilon=epsilon)
                
                _y_train = y_train if not y_origin else std['rGFR']*y_train + mean['rGFR']
                _y_test  = y_test  if not y_origin else std['rGFR']*y_test  + mean['rGFR']
                
                model.fit(X_train, _y_train)
                
                _y_train_pred = model.predict(X_train)
                _y_test_pred =  model.predict(X_test)

                train_loss = np.mean((_y_train - _y_train_pred)**2)
                test_loss = np.mean((_y_test - _y_test_pred)**2)
                
                if not y_origin:
                    train_loss_re = np.mean(((std['rGFR']*_y_train_pred + mean['rGFR']) - (std['rGFR']*_y_train + mean['rGFR']))**2)
                    test_loss_re =  np.mean(((std['rGFR']*_y_test_pred + mean['rGFR']) - (std['rGFR']*_y_test + mean['rGFR']))**2)
                else:
                    train_loss_re = np.nan
                    test_loss_re = np.nan
                
                if verbose:
                    print('ga = {} C = {} eps = {} tr_l ={:.4f} te_l = {:.4f} tr_lr = {:.4f} te_lr = {:.4f}'.format(gamma, C, epsilon, \
                            train_loss, test_loss, train_loss_re, test_loss_re))
                r_list.append([gamma, C, epsilon, train_loss, test_loss, train_loss_re, test_loss_re])
                
                mat[i,j,k,0] = train_loss
                mat[i,j,k,1] = test_loss
                mat[i,j,k,2] = train_loss_re
                mat[i,j,k,3] = test_loss_re
    
    return mat,r_list

In [47]:
mat,r_list= grid_search(gamma_list = [0.1,1.0, 10.0, 'auto'], C_list = [0.1,1.0,10], epsilon_list = [0.05,0.1,0.15])

ga = 0.1 C = 0.1 eps = 0.05 tr_l =0.2690 te_l = 0.2509 tr_lr = 226.3662 te_lr = 211.1428
ga = 0.1 C = 0.1 eps = 0.1 tr_l =0.2689 te_l = 0.2505 tr_lr = 226.2250 te_lr = 210.7486
ga = 0.1 C = 0.1 eps = 0.15 tr_l =0.2697 te_l = 0.2513 tr_lr = 226.9285 te_lr = 211.4691
ga = 0.1 C = 1.0 eps = 0.05 tr_l =0.2382 te_l = 0.2224 tr_lr = 200.4690 te_lr = 187.1667
ga = 0.1 C = 1.0 eps = 0.1 tr_l =0.2379 te_l = 0.2223 tr_lr = 200.1583 te_lr = 187.0890
ga = 0.1 C = 1.0 eps = 0.15 tr_l =0.2378 te_l = 0.2230 tr_lr = 200.1244 te_lr = 187.6227
ga = 0.1 C = 10 eps = 0.05 tr_l =0.2334 te_l = 0.2195 tr_lr = 196.3819 te_lr = 184.6538
ga = 0.1 C = 10 eps = 0.1 tr_l =0.2331 te_l = 0.2207 tr_lr = 196.1441 te_lr = 185.7432
ga = 0.1 C = 10 eps = 0.15 tr_l =0.2335 te_l = 0.2235 tr_lr = 196.5160 te_lr = 188.0779
ga = 1.0 C = 0.1 eps = 0.05 tr_l =0.2561 te_l = 0.2354 tr_lr = 215.4772 te_lr = 198.0930
ga = 1.0 C = 0.1 eps = 0.1 tr_l =0.2564 te_l = 0.2356 tr_lr = 215.7697 te_lr = 198.2329
ga = 1.0 C = 0.1 eps = 0.15 

In [48]:
sorted(r_list, key=lambda x:x[-1])[:3]

[[0.1,
  10,
  0.05,
  0.23338876731633734,
  0.21945056205201657,
  196.38188492706252,
  184.65376684416628],
 [0.1,
  10,
  0.1,
  0.23310618498669694,
  0.2207452582731141,
  196.14410977112863,
  185.74317181952432],
 ['auto',
  1.0,
  0.05,
  0.23418236636452125,
  0.2214842378536577,
  197.04964832780735,
  186.36497639314837]]

In [50]:
mat = grid_search(gamma_list = [0.05,0.5, 1.0, 'auto'], C_list = [1.0,5.0,1.0], epsilon_list = [0.05,0.1,0.15])
sorted(r_list, key=lambda x:x[-1])[:3]

ga = 0.05 C = 1.0 eps = 0.05 tr_l =0.2475 te_l = 0.2286 tr_lr = 208.2496 te_lr = 192.3305
ga = 0.05 C = 1.0 eps = 0.1 tr_l =0.2485 te_l = 0.2303 tr_lr = 209.0988 te_lr = 193.7863
ga = 0.05 C = 1.0 eps = 0.15 tr_l =0.2493 te_l = 0.2306 tr_lr = 209.8098 te_lr = 194.0497
ga = 0.05 C = 5.0 eps = 0.05 tr_l =0.2386 te_l = 0.2213 tr_lr = 200.8015 te_lr = 186.2492
ga = 0.05 C = 5.0 eps = 0.1 tr_l =0.2380 te_l = 0.2222 tr_lr = 200.2694 te_lr = 186.9499
ga = 0.05 C = 5.0 eps = 0.15 tr_l =0.2378 te_l = 0.2222 tr_lr = 200.1209 te_lr = 186.9991
ga = 0.05 C = 1.0 eps = 0.05 tr_l =0.2475 te_l = 0.2286 tr_lr = 208.2496 te_lr = 192.3305
ga = 0.05 C = 1.0 eps = 0.1 tr_l =0.2485 te_l = 0.2303 tr_lr = 209.0988 te_lr = 193.7863
ga = 0.05 C = 1.0 eps = 0.15 tr_l =0.2493 te_l = 0.2306 tr_lr = 209.8098 te_lr = 194.0497
ga = 0.5 C = 1.0 eps = 0.05 tr_l =0.2319 te_l = 0.2240 tr_lr = 195.0901 te_lr = 188.4747
ga = 0.5 C = 1.0 eps = 0.1 tr_l =0.2315 te_l = 0.2235 tr_lr = 194.7670 te_lr = 188.0462
ga = 0.5 C = 1.0

[[0.1,
  10,
  0.05,
  0.23338876731633734,
  0.21945056205201657,
  196.38188492706252,
  184.65376684416628],
 [0.1,
  10,
  0.1,
  0.23310618498669694,
  0.2207452582731141,
  196.14410977112863,
  185.74317181952432],
 ['auto',
  1.0,
  0.05,
  0.23418236636452125,
  0.2214842378536577,
  197.04964832780735,
  186.36497639314837]]

这个大致就定成0.1, 10, 0.05了。 test loss 184

另外y的尺度要不要标准化是有点疑问的。不要说y若标准化会给后续的处理带来很多不一致的地方。另外在标准化的情况下对数化就不能直接进行了，
是先对数化再标准化吗。

先试试看y用未标准化版本会怎么样。

In [56]:
mat = grid_search(gamma_list = [0.1,1.0, 10.0, 'auto'], C_list = [0.1,1.0,10], epsilon_list = [0.05,0.1,0.15], y_origin=True)

ga = 0.1 C = 0.1 eps = 0.05 tr_l =534.6323 te_l = 503.7067 tr_lr = nan te_lr = nan
ga = 0.1 C = 0.1 eps = 0.1 tr_l =535.1436 te_l = 504.2476 tr_lr = nan te_lr = nan
ga = 0.1 C = 0.1 eps = 0.15 tr_l =535.1528 te_l = 504.3767 tr_lr = nan te_lr = nan
ga = 0.1 C = 1.0 eps = 0.05 tr_l =255.3818 te_l = 242.0037 tr_lr = nan te_lr = nan
ga = 0.1 C = 1.0 eps = 0.1 tr_l =255.3925 te_l = 242.1054 tr_lr = nan te_lr = nan
ga = 0.1 C = 1.0 eps = 0.15 tr_l =255.4119 te_l = 242.1528 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.05 tr_l =206.5622 te_l = 190.8897 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.1 tr_l =206.5720 te_l = 190.9080 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.15 tr_l =206.4706 te_l = 190.8523 tr_lr = nan te_lr = nan
ga = 1.0 C = 0.1 eps = 0.05 tr_l =596.2775 te_l = 544.3851 tr_lr = nan te_lr = nan
ga = 1.0 C = 0.1 eps = 0.1 tr_l =595.8391 te_l = 543.8714 tr_lr = nan te_lr = nan
ga = 1.0 C = 0.1 eps = 0.15 tr_l =595.6980 te_l = 543.5691 tr_lr = nan te_lr = nan
ga = 1.0 C 

In [57]:
mat = grid_search(gamma_list = [0.1,1.0, 10.0, 'auto'], C_list = [1.0,10, 100], epsilon_list = [0.05,0.1,0.15], y_origin=True)

ga = 0.1 C = 1.0 eps = 0.05 tr_l =255.3818 te_l = 242.0037 tr_lr = nan te_lr = nan
ga = 0.1 C = 1.0 eps = 0.1 tr_l =255.3925 te_l = 242.1054 tr_lr = nan te_lr = nan
ga = 0.1 C = 1.0 eps = 0.15 tr_l =255.4119 te_l = 242.1528 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.05 tr_l =206.5622 te_l = 190.8897 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.1 tr_l =206.5720 te_l = 190.9080 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.15 tr_l =206.4706 te_l = 190.8523 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.05 tr_l =197.4144 te_l = 184.6554 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.1 tr_l =197.4497 te_l = 184.9185 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.15 tr_l =197.4382 te_l = 184.9645 tr_lr = nan te_lr = nan
ga = 1.0 C = 1.0 eps = 0.05 tr_l =258.8592 te_l = 230.8444 tr_lr = nan te_lr = nan
ga = 1.0 C = 1.0 eps = 0.1 tr_l =258.5981 te_l = 230.5372 tr_lr = nan te_lr = nan
ga = 1.0 C = 1.0 eps = 0.15 tr_l =258.4042 te_l = 230.3476 tr_lr = nan te_lr = nan
ga = 1.0 C 

In [58]:
mat = grid_search(gamma_list = [0.1,1.0, 10.0, 'auto'], C_list = [10, 100, 1000], epsilon_list = [0.05,0.1,0.15], y_origin=True)

ga = 0.1 C = 10 eps = 0.05 tr_l =206.5622 te_l = 190.8897 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.1 tr_l =206.5720 te_l = 190.9080 tr_lr = nan te_lr = nan
ga = 0.1 C = 10 eps = 0.15 tr_l =206.4706 te_l = 190.8523 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.05 tr_l =197.4144 te_l = 184.6554 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.1 tr_l =197.4497 te_l = 184.9185 tr_lr = nan te_lr = nan
ga = 0.1 C = 100 eps = 0.15 tr_l =197.4382 te_l = 184.9645 tr_lr = nan te_lr = nan
ga = 0.1 C = 1000 eps = 0.05 tr_l =196.0130 te_l = 184.2467 tr_lr = nan te_lr = nan
ga = 0.1 C = 1000 eps = 0.1 tr_l =196.0677 te_l = 184.3440 tr_lr = nan te_lr = nan
ga = 0.1 C = 1000 eps = 0.15 tr_l =195.9795 te_l = 184.1494 tr_lr = nan te_lr = nan
ga = 1.0 C = 10 eps = 0.05 tr_l =196.9122 te_l = 192.9644 tr_lr = nan te_lr = nan
ga = 1.0 C = 10 eps = 0.1 tr_l =196.9876 te_l = 192.8643 tr_lr = nan te_lr = nan
ga = 1.0 C = 10 eps = 0.15 tr_l =197.0641 te_l = 192.9396 tr_lr = nan te_lr = nan
ga = 1.0 C 

In [59]:
mat = grid_search(gamma_list = [0.05,0.1, 0.15, 'auto'], C_list = [50, 100, 150], epsilon_list = [0.05,0.1,0.15], y_origin=True)

ga = 0.05 C = 50 eps = 0.05 tr_l =205.2061 te_l = 189.4933 tr_lr = nan te_lr = nan
ga = 0.05 C = 50 eps = 0.1 tr_l =205.1569 te_l = 189.4432 tr_lr = nan te_lr = nan
ga = 0.05 C = 50 eps = 0.15 tr_l =205.0885 te_l = 189.4444 tr_lr = nan te_lr = nan
ga = 0.05 C = 100 eps = 0.05 tr_l =201.3309 te_l = 186.3805 tr_lr = nan te_lr = nan
ga = 0.05 C = 100 eps = 0.1 tr_l =201.3932 te_l = 186.4354 tr_lr = nan te_lr = nan
ga = 0.05 C = 100 eps = 0.15 tr_l =201.4319 te_l = 186.4868 tr_lr = nan te_lr = nan
ga = 0.05 C = 150 eps = 0.05 tr_l =199.7808 te_l = 184.8358 tr_lr = nan te_lr = nan
ga = 0.05 C = 150 eps = 0.1 tr_l =199.7707 te_l = 184.8665 tr_lr = nan te_lr = nan
ga = 0.05 C = 150 eps = 0.15 tr_l =199.7931 te_l = 184.8836 tr_lr = nan te_lr = nan
ga = 0.1 C = 50 eps = 0.05 tr_l =199.0729 te_l = 186.1583 tr_lr = nan te_lr = nan
ga = 0.1 C = 50 eps = 0.1 tr_l =199.0498 te_l = 186.3103 tr_lr = nan te_lr = nan
ga = 0.1 C = 50 eps = 0.15 tr_l =199.0376 te_l = 186.3903 tr_lr = nan te_lr = nan
ga = 

不出意料只要允许C取大，其实能得到差不多的效果而不需要标准化因变量。

ga = 0.1 C = 100 eps = 0.05 tr_l =197.4144 te_l = 184.6554 tr_lr = nan te_lr = nan

不过说起来这也只不过达到了184.6554，之前虽然不做对数变换的达不到这个，但是做了对数变化的完全可以达到这个。。

如果是先对数变换再标准化的话